# Outline
- Pre-processing
- Additive model
- Dendrogram/PC
- Multiplicitive model

# Set Environment

In [ ]:
source("course_config.R")
source("../../pilot/pilot_util.R")

# Import Data & Create DESeq from Count Data

In [ ]:
# import RData (annomapres0, annogenecnts0)
attach(file.path(OUTDIR, "HTS-Course-Annotated-STAR-counts.RData"))

Lets review what we have created before

In [ ]:
# metadata
head(annomapres0, 3)

In [ ]:
# count matrix
head(annogenecnts0, 3)

Prepare columnData DataFrame and countData (matrix object)
- columnData --- metadata
- countData  --- count matrix

In [ ]:
# columnData --- metadata
annomapres0 %>%
    DataFrame ->
    columnData
rownames(columnData) <- columnData[["Label"]]

head(columnData[, c("Label", "Strain", "Media")], 3)

In [ ]:
# countData  --- count matrix
annogenecnts0 %>%
    dplyr::select(dput(as.character(c("gene", columnData[["Label"]])))) %>%
    as.data.frame %>%
    column_to_rownames("gene") %>%
    as.matrix ->
    countData

head(countData, 3)

### Make DESeq object on the basis of the counts

The design option allows you to specify an additive or a multiplicitive model

Additive model

In [ ]:
dds_add <- DESeqDataSetFromMatrix(
    countData,                       # Count matrix
    columnData,                      # metadata
    ~ Media + Strain) # design formula

Multiplicitive model

In [ ]:
dds_mult <- DESeqDataSetFromMatrix(
    countData,                       # Count matrix
    columnData,                      # metadata
    ~ Media + Strain + Media:Strain) # design formula

In the following demonstration, we will use the additive model. The multiplicitive model will be illustrated in the appendix below.

In [ ]:
dds <- dds_add

# Inspect object & Slots of an S4 class

Let's has a look at the object we have created.

In [ ]:
dds

see the class of dds object

In [ ]:
class(dds)

DESeqDataSet is a S4 object. Recall that a S4 object was taught when introducing bioconductor. Note that S4 objects allow users to wrap up multiple elements into a single variables where each element is called a slot.

In [ ]:
slotNames(dds)

The metadata (columnData) is stored in the slot `colData`

In [ ]:
dds@colData %>% as.data.frame %>% head(3)

The design formula is stored in the slot `design`. The design holds the R formula which expresses how the counts depend on the variables in colData.

In [ ]:
dds@design

The first thing you may want to do is **have a look at the raw counts** you have imported. The `DESeq2::counts` function extracts a matrix of counts (with the genes along the rows and samples along the columns). Let us first verify the dimension of this matrix.

In [ ]:
dim(counts(dds))

In [ ]:
head(counts(dds),3)

This slot returns gene specific information (it will be populated later)

In [ ]:
dds@dispersionFunction

# Estimate Size Factors and Dispersion Parameters

You recall that DESeq requires that  we have estimates for sample specific size factors and gene specific dispersion factors. More specifically, recall that DESeq models the count $K_{ij}$ (gene $i$, sample $j$) as negative binomial with mean $\mu_{ij}$ and dispersion parameter $\alpha_i$. Here $\mu_{ij}=s_j q_{ij}$ where $\log_2(q_{ij}) = \beta_{0i} + \beta_{1i} z_j$. Here $s_j$ is the sample $j$ specific size factor.

**Summarize of notation**
- $K_{ij}$ denotes the observed **number of reads** mapped to gene $i$ for sample $j$
- $K_{ij}$ follows a **negative binomial distribution** with
    - **Mean** $\mu_{ij}$
    - **Dispersion parameter** $\alpha_i$
- Modelling
    - $K_{ij} \sim NB(\mu_{ij}, \alpha_i)$
    - $\mu_{ij} = s_{j}q_{ij}$
        - $s_j$ is sample $j$ specific normalization constant
    - $\log_2(q_{ij}) = \beta_{0i} + \beta_{1i} z_j$

## 01 Size Factors
 We begin by estimating the size factors $s_1,\ldots,s_n$:

In [ ]:
dds <- estimateSizeFactors(dds)

Now, compare the dds object to that of before applying the estimateSizeFactors() function. What has changed? What remains unchanged?

In [ ]:
dds

Note that there is a **sizeFactor** added to **colData**. Let's look at it more carefully

```
> dds # (before estimateSizeFactors)
class: DESeqDataSet 
dim: 8497 24 
metadata(1): version
assays(1): counts
rownames(8497): CNAG_00001 CNAG_00002 ... ENSRNA049551964 ENSRNA049551993
rowData names(0):
colnames(24): 1_RZ_J 10_RZ_C ... 47_RZ_P 9_RZ_C
colData names(10): Label Strain ... prob.unique depth

> dds # (after estimateSizeFactors)
class: DESeqDataSet 
dim: 8497 24 
metadata(1): version
assays(1): counts
rownames(8497): CNAG_00001 CNAG_00002 ... ENSRNA049551964 ENSRNA049551993
rowData names(0):
colnames(24): 1_RZ_J 10_RZ_C ... 47_RZ_P 9_RZ_C
colData names(11): Label Strain ... depth sizeFactor <------ 
```

You can also get the size factors directly

In [ ]:
sizeFactors(dds)

 It is preferable to limit the number of decimal places. Next show the size factors rounded to 3 decimal places

In [ ]:
round(sizeFactors(dds),3)

Now that the size factors have been estimated, we can get "normalized" counts. Here we print three data frames together to easily compare them.

In [ ]:
# original count
head(counts(dds),3)

# normalized count
head(counts(dds, normalize = TRUE), 3)

# normalized manually using size factors
counts(dds)[1:3,] %>% 
    apply(., 1, function(row){row / sizeFactors(dds)}) %>%
    t

**Exercise:** How do you get the raw counts for gene  "GeneID: CNAG_05845"?

In [ ]:
counts(dds, normalize = TRUE)["CNAG_05845",]

**Exercise:** Get a summary (mean, median, quantiles etc ) of the size factors

In [ ]:
summary(sizeFactors(dds))

Before going to the next step, let's look at the dispersionFunction slot

In [ ]:
# still empty
dds@dispersionFunction

## 02 Dispersion Parameters
Next, we get the dispersion factors $\alpha_1,\ldots,\alpha_{m}$

In [ ]:
dds <- estimateDispersions(dds)

Now inspect the dds object again and note that the rowRanges slot has extra information ("metadata column names(0):" before versus "column names(9): baseMean baseVar ... dispOutlier dispMAP")
- before: 
    - `metadata column names(0):`
- after:  
    - `column names(9): baseMean baseVar ...`

In [ ]:
dds

Can you notice the difference?
```
> dds (before dispersion)
class: DESeqDataSet 
dim: 8497 24 
metadata(1): version
assays(1): counts
rownames(8497): CNAG_00001 CNAG_00002 ... ENSRNA049551964 ENSRNA049551993
rowData names(0):
colnames(24): 1_RZ_J 10_RZ_C ... 47_RZ_P 9_RZ_C
colData names(11): Label Strain ... depth sizeFactor

> dds (after dispersion)
class: DESeqDataSet 
dim: 8497 24 
metadata(1): version
assays(2): counts mu
rownames(8497): CNAG_00001 CNAG_00002 ... ENSRNA049551964 ENSRNA049551993
rowData names(9): baseMean baseVar ... dispOutlier dispMAP <------
colnames(24): 1_RZ_J 10_RZ_C ... 47_RZ_P 9_RZ_C
colData names(11): Label Strain ... depth sizeFactor
```

Note that the dispersionfunction slot is now populated

In [ ]:
dds@dispersionFunction

We can extract the gene specific dispersion factors using dispersions(). Note that there will be one number per gene. We look at the first four genes (rounded to 4 decimal places)

In [ ]:
alphas <- dispersions(dds)

Verify that the number of dispersion factors equals the number of genes

In [ ]:
# number of disperion factors
length(alphas)

In [ ]:
# number of genes
nrow(dds)

Print the dispersion factors for the first four genes rounded to four decimal points

In [ ]:
round(alphas[1:4], 4)

Extract the metadata using mcols() for the first four genes

| Terms       | Description                                   |
|-------------|-----------------------------------------------|
| baseMean    |     mean of normalized counts for all samples |
| baseVar     | variance of normalized counts for all samples |
| allZero     |                all counts for a gene are zero |
| dispGeneEst |             gene-wise estimates of dispersion |
| dispFit     |                   fitted values of dispersion |
| dispersion  |                  final estimate of dispersion |
| dispIter    |                          number of iterations |
| dispOut     |                 dispersion flagged as outlier |
| dispMAP     |                 maximum a posteriori estimate |


In [ ]:
mcols(dds)[1:4,] %>% as.data.frame

**Exercise:** Provide statistical summaries of the dispersion factors

In [ ]:
summary(dispersions(dds))

**Exercise:** Summarize the dispersion factors using a box plot (may want to log transform)

In [ ]:
boxplot(log(dispersions(dds)))

# Differential Expression Analysis
We can now conduct a differential expression analysis using the DESeq() function. Keep in mind that to get to this step, we first estimated the size factors and then the dispersion parameters.

In [ ]:
ddsDE <- DESeq(dds)

Note that currently, the model we have is an additive model, which does not include the interaction term of `Media` and `Strain`

In [ ]:
design(dds)

We can get the results for the differential expression analysis using results(). Here, we can compare two group of samples specified by the contrast. (If not, the default contrast would be the last term in your additive model `design(dds)`).

In [ ]:
# compare two Media
myres_media <- results(ddsDE, contrast = c("Media", "YPD", "TC"))

In [ ]:
# compare two Strains
myres_strain <- results(ddsDE, contrast = c("Strain", "H99", "mar1d"))

Let's look at the results for the first four genes

In [ ]:
# first four genes
myres_strain[1:4,]

 You can get the descriptions for the columns from the DE analysis

In [ ]:
data.frame(desc = mcols(myres_strain)$description) 

manually calculate the baseMean to see if a gene

In [ ]:
t(counts(dds, normalize = TRUE)["CNAG_00001",])

In [ ]:
mean(counts(dds, normalize = TRUE)["CNAG_00001",])

##  P-values

Here we will play with the p-value of the results. Below we demonstrate how the p-value is adjusted using BH method

One can extract the unadjusted p-values as follows

In [ ]:
pvalues <- myres_strain$pvalue
length(pvalues)
pvalues[1:4]

The BH adjusted p-values can be extracted as

In [ ]:
adjp <- myres_strain$padj
length(adjp)
adjp[1:4]

Calculate BH adjusted P-values by "hand" using the p.adjust() function. Note that you will not replicate the results you get under the padj column (when looking at the first four rows)

In [ ]:
pvalues <- myres_strain$pvalue
BH <- p.adjust(pvalues,"BH")
data.frame(BH = BH[1:4], adjp = adjp[1:4])

The DESeq2::results function applies "independent" filtering. This is enabled by default. Let's disable and then reexamine the adjusted P-values

In [ ]:
myres1 <- results(ddsDE, independentFiltering = FALSE)

In [ ]:
myres1

 We can now replicate the results

In [ ]:
pvalues1 <- myres1$pvalue
BH1 <- p.adjust(pvalues1[!is.na(pvalues)], "BH")
data.frame(
    BH   = BH1[1:4],
    adjp = myres1$padj[1:4])

##  Subset and reorder the results

In [ ]:
class(myres_strain)

In [ ]:
summary(myres_strain, 0.05)

 You can sort the results by say the unadjusted P-values

In [ ]:
results(ddsDE, contrast = c("Strain", "H99", "mar1d"), tidy = TRUE) %>%
    arrange(padj) %>% 
    head(4)

To get the list of genes with unadjusted P-values < 0.00001 and absolute log2 FC of more than 4

In [ ]:
results(ddsDE, contrast = c("Strain", "H99", "mar1d"), tidy = TRUE) %>%
    filter(padj < 0.00001) %>%
    filter(abs(log2FoldChange) > 4)

The P-values for the four top genes are beyond machine precision. You can use the format.pval() function to properly format the P-values. PLEASE promote ending the practice of publishing P-values below machine precision.  (that would be akin to stating the weight of an object that weighs less than one pound with a scale whose minimum weight spec is 1 pound).

In [ ]:
results(ddsDE, contrast = c("Strain", "H99", "mar1d"), tidy = TRUE) %>%
    filter(padj < 0.00001) %>%
    filter(abs(log2FoldChange) > 4) %>%
    mutate(pval = format.pval(pvalue))

Let's look at a volcano plot

In [ ]:
plot(myres_strain$log2FoldChange,
     -log10(myres_strain$padj),
     pch  = 19, 
     cex  = 0.3,
     xlab = "Log2 FC",
     ylab = "-log10(BH Adjusted P-value)")

Exercise: Annotate the hits with adjusted P-values < 0.05 and absolute log2 FC greater than 2 in red

In [ ]:
plot(myres_strain$log2FoldChange,
     -log10(myres_strain$padj),
     pch  = 19,
     cex  = 0.3,
     xlab = "Log2 FC",
     ylab = "-log10(BH Adjusted P-value)",
     col  = ifelse(myres_strain$padj < 0.05 & abs(myres_strain$log2FoldChange) > 2,
                   "red",
                   "black"))

# Converting/Normalizing Counts to "Expressions"

##  Normalized Counts
We have already shown how to "normalize" the counts using the estimated size factors

In [ ]:
head(counts(dds, normalize = TRUE), 3)

Plot the counts stratified by treatment for the 2nd gene. Later we will compare the expression values in more detail in the section of regularized log transformation (rlog transformation). In the section of rlog transformation, the media and strain will be indicated/labeled in the plot.

In [ ]:
results(ddsDE, contrast = c("Strain", "H99", "mar1d"), tidy = TRUE) %>%
    arrange(padj) %>% 
    head(4)

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 4)
par(mfrow = c(1, 2))
plotCounts(dds, 2,            intgroup = "Strain")
plotCounts(dds, "CNAG_03398", intgroup = "Strain")
par(mfrow = c(1, 1))

In [ ]:
results(ddsDE, contrast = c("Media", "YPD", "TC"), tidy = TRUE) %>%
    arrange(padj) %>% 
    head(4)

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 4)
par(mfrow = c(1, 2))
plotCounts(dds, 2,            intgroup = "Media")
plotCounts(dds, "CNAG_03398", intgroup = "Media")
par(mfrow = c(1, 1))

#  FPM
Another approach is to FPM: fragments per million mapped fragments

In [ ]:
head(fpm(dds), 3)

Let's calculate the FPM manually. For gene $i$ sample $j$, the FPM is defined as $\frac{K_{ij}}{D_j}\times 10^{6}$ where $D_j=\sum_{i=1} K_{ij}$ is the read depth for sample $j$. First get the read depth for each sample

In [ ]:
D <- colSums(counts(dds)) # colSums: sum of each column, where each column represents a sample
D

By default, the fpm() function uses a robust approach. We will disable this right now as to replicate the standard FPM. Let's look at gene 1

In [ ]:
fpm1 <- fpm(dds, robust = FALSE)[1,]
fpm1

Now get the raw counts for gene 1

In [ ]:
cnt1 <- counts(dds)[1,]
cnt1

 Now calculate the FPM for gene 1

In [ ]:
myfpm1 <- cnt1 / D * 1e6
myfpm1

Let's summarize what we have done:

In [ ]:
tmp <- bind_rows(D, cnt1, myfpm1, fpm1)
tmp <- t(tmp)
colnames(tmp) <- c("col_sums", "count_gene1", "FPM_gene1_manual", "FPM_gene1_DESeq")
tmp

This is how you check if two numeric columns are "equal"? One approach is to calculate the maximum absoute difference

In [ ]:
max(abs(fpm1 - myfpm1))

The above approach is also helpful in establishing if the difference is "small". Another approach to test for equality to use the all.equal() function

In [ ]:
all.equal(fpm1, myfpm1)

 It is generally a bad idea to compare numeric vectors using == (e.g., fpm1==myfpm1)

### FPKM

 To calculate the FPKM (fragments per kilobase per million mapped fragments) we need to add annotation to assign the feature lengths. More specifically, for gene $i$ sample $j$, the FPKM is defined as $\frac{K_{ij}}{\ell_i D_j}\times 10^3 \times 10^{6}$ where $\ell_i$ is the "length" of gene $i$ (fragments for each $10^3$ bases in the gene for every  $\frac{D_j}{10^6}$ fragments. More on this later.

# Regularized log transformation
The regularized log transform can be obtained using the [rlog() function](https://rdrr.io/bioc/DESeq2/man/rlog.html). Note that an important argument for this function is blind (TRUE by default). The default "blinds" the normalization to the design. This is very important so as to not bias the analyses (e.g. class discovery) 

In [ ]:
rld <- rlog(dds, blind = TRUE)

## Dendrogram of samples: showing strain & media of each sample

Hierarchical clustering using rlog transformation

In [ ]:
options(repr.plot.width = 9, repr.plot.height = 5)
dists <- dist(t(assay(rld)))
plot(hclust(dists)) 

Store the dendrogram of samples using hierarchical clustering

In [ ]:
assay(rld) %>%
    t() %>%
    dist %>%
    hclust(method = "complete") %>%
    as.dendrogram ->
    mydend

Dendrogram of samples: showing strain of each sample

In [ ]:
options(repr.plot.width = 9, repr.plot.height = 5)
dendplot(mydend, columnData, 
         "Strain",    # variable that show in label
         "Strain",    # variable that define color
         "Media") %>% # variable that define shape of points
    plot

Dendrogram of samples: showing media of each sample

In [ ]:
dendplot(mydend, columnData, "Media", "Strain", "Media") %>% plot

Dendrogram of samples: showing sample label

In [ ]:
dendplot(mydend, columnData, "Label", "Media", "Strain") %>% plot

PC Analysis using the rlog transformation

In [ ]:
library(ggplot2)
options(repr.plot.width = 7, repr.plot.height = 4)
plotPCA(rld, intgroup = "Media") + geom_text(label=columnData$Label, color="black")

Store the plot into a pdf

In [ ]:
pdf(file.path(IMGDIR, "dendrogram.pdf"))
dendplot(mydend, columnData, "Strain", "Strain", "Media") %>% plot
dendplot(mydend, columnData, "Media", "Strain", "Media") %>% plot
dendplot(mydend, columnData, "Label", "Strain", "Media") %>% plot
graphics.off()

# Variance Stabilizing Transformation (vst) and mean-variance modelling at the observational level (voom)

 Two other normalization approaches for RNA-Seq count data are provided by the functions DESeq2::vst and limma::voom (note that for the latter one needs the limma package).From ? DESeq2::vst
"This function calculates a variance stabilizing transformation (VST) from the fitted dispersion-mean relation(s) and then transforms the count data (normalized by division by the size factors or normalization factors), yielding a matrix of values which are now approximately homoskedastic (having constant variance along the range of mean values). The transformation also normalizes with respect to library size."Compared to DESeq2::rlog
"The ‘rlog’ is less sensitive to size factors, which can be an issue when size factors vary widely. These transformations are useful when checking for outliers or as input for machine learning techniques such as clustering or linear discriminant analysis."From ? limma::voom
"Transform count data to log2-counts per million (logCPM), estimate the mean-variance relationship and use this to compute appropriate observation-level weights. The data are then ready for linear modelling." Get VST transformation

In [ ]:
VST <- vst(dds)
class(VST)

Get the VST matrix

In [ ]:
VSTmat <- assay(VST)
dim(VSTmat)
VSTmat[1:10,]

Get voom transformation (note that according to ? limma::voom, the function is expecting raw counts

In [ ]:
VOOM <- limma::voom(counts(dds))

Get the VOOM matrix

In [ ]:
VOOMmat <- VOOM$E
dim(VOOMmat)
VOOMmat[1:10,]

# Appendix: Multiplicitive Model

In [ ]:
### Make DESeq object on the basis of the counts
dds_mult <- DESeqDataSetFromMatrix(countData, columnData, ~ Media + Strain + Media:Strain)
### Estimate Size Factors
dds_mult <- estimateSizeFactors(dds_mult)
### Estimate Dispersion parameters (for each gene)
dds_mult <- estimateDispersions(dds_mult)
### Fit NB MLE model
dds_mult <- DESeq(dds_mult)
### Rlog "normalized" expressions
rld <- rlog(dds_mult)

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 5)
grid.arrange(
    myinteractplot(rld, "CNAG_03398", "Strain"),
    myinteractplot(rld, "CNAG_03623", "Strain"),
    myinteractplot(rld, "CNAG_00727", "Strain"),
    myinteractplot(rld, "CNAG_02587", "Strain"),
    ncol=2)

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 5)
grid.arrange(
    myinteractplot(rld, "CNAG_03398", "media"),
    myinteractplot(rld, "CNAG_12988", "media"),
    myinteractplot(rld, "CNAG_00183", "media"),
    myinteractplot(rld, "CNAG_02083", "media"),
    ncol=2)

# Appendix: Get Session Information

In [ ]:
sessionInfo()